# Advent of Code 2025
## Day 10
### Part 1

In [61]:
import re

all_squares = []
all_parens = []
all_curly = []

with open("day10.txt", "r") as f:
    for line in f:
        line = line.strip()

        # Extract square brackets content
        square_raw = re.search(r'\[(.*?)\]', line).group(1)
        square = list(square_raw)

        # Extract parentheses content → list of lists
        parens_raw = re.findall(r'\((.*?)\)', line)
        parens = [
            list(map(int, group.split(',')))
            for group in parens_raw
        ]

        # Extract curly braces content
        curly_raw = re.search(r'\{(.*?)\}', line).group(1)
        curly = list(map(int, curly_raw.split(',')))

        # Store results
        all_squares.append(square)
        all_parens.append(parens)
        all_curly.append(curly)

I convert both the lights setup and the buttons to binary representation. One thing to note here is that no button needs to be pressed more than once as pressing a button twice is the same as not pressing it at all. Then, I iterate through all the possible combinations starting with just pushing 1 button, then 2 buttons ... This is done until a solution is found which is guaranteed to be the minimum button press as we started from 1.

In [86]:
def transform_to_binary(buttons, lights):
    transformed_buttons = []

    for button in buttons:
        temp_binary = list("0" * len(lights))
        for press in button:
            temp_binary[press] = "1"
        transformed_buttons.append(''.join(temp_binary))

    lights_temp = lights.copy()
    for i, light in enumerate(lights):
        
        if light == ".":
            lights_temp[i] = "0"
        else:
            lights_temp[i] = "1"
        transformed_lights = ''.join(lights_temp)
        
    return transformed_buttons, transformed_lights

import itertools
def solve_minimal_toggles(options_list, target_state):
    # Convert binary strings to integers for easier bitwise operations
    options_int = [int(o, 2) for o in options_list]
    target_int = int(target_state, 2)
    num_options = len(options_int)

    # Iterate through combinations by size, starting from the smallest (1 option)
    for num_toggles in range(1, num_options + 1):
        # Generate all combinations of 'num_toggles' options
        for combo_indices in itertools.combinations(range(num_options), num_toggles):
            current_state = 0
            # Apply (XOR) the selected options
            for index in combo_indices:
                current_state ^= options_int[index]
            
            # Check if this combination matches the target
            if current_state == target_int:
                # Solution found! Return the indices of the options used
                option_names = [f"Option {i} ({options_list[i]})" for i in combo_indices]
                return num_toggles, option_names, current_state

    return 0, [], 0 # No solution found


In [87]:
runsum = 0
for lights, buttons in zip(all_squares, all_parens):
    buttons_binary, lights_binary = transform_to_binary(buttons, lights)
    num_toggles, option_names, current_state = solve_minimal_toggles(buttons_binary, lights_binary)
    runsum += num_toggles
print(runsum)

455


### Part 2

In [ ]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

def solve_ilp_minimal_sum(options_list, target_state_list):
    #convert binary strings to lists of integers [0, 1, 0, 1]
    options_matrix_rows = []
    for opt_str in options_list:
        options_matrix_rows.append([int(bit) for bit in opt_str])
    

    A = np.array(options_matrix_rows).T
    # Define the Target Vector 'B'
    b = np.array(target_state_list)
    num_options = A.shape[1]
    num_bits = A.shape[0] 
    #Define the Objective Function (Minimize the sum of coefficients b_i)
    c = np.ones(num_options) 

    #Define Constraints
    constraints = [LinearConstraint(A, b, b)] 
    # The coefficients 'x' must be non-negative integers
    bounds = Bounds(np.zeros(num_options), np.full(num_options, np.inf))
    
    #Define Integrality (all variables must be integers)
    integrality = np.ones(num_options) # 1 means integer, 0 means continuous

    #Solve the optimization problem
    res = milp(c, constraints=constraints, bounds=bounds, integrality=integrality)

    if res.success:
        return res.x, res.fun
    else:
        return None, "No solution found"

In [91]:
runsum = 0
for lights, buttons, joltages in zip(all_squares, all_parens, all_curly):
    buttons_binary, lights_binary = transform_to_binary(buttons, lights)
    coefs, objective_fun = solve_ilp_minimal_sum(buttons_binary, joltages)
    runsum += objective_fun
print(runsum)

16978.0
